In [162]:
%matplotlib inline  
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import matplotlib.pyplot as plt
import pylab
import seaborn as sns

In [194]:
data_0 = pd.read_csv('Kickstarter000.csv')
data_1 = pd.read_csv('Kickstarter001.csv')
data_2 = pd.read_csv('Kickstarter002.csv')
data_3 = pd.read_csv('Kickstarter003.csv')
data_4 = pd.read_csv('Kickstarter004.csv')
data_5 = pd.read_csv('Kickstarter005.csv')
data_6 = pd.read_csv('Kickstarter006.csv')
data_7 = pd.read_csv('Kickstarter007.csv')
data_8 = pd.read_csv('Kickstarter008.csv')
data_9 = pd.read_csv('Kickstarter009.csv')
data_10 = pd.read_csv('Kickstarter010.csv')
data_11 = pd.read_csv('Kickstarter011.csv')

In [195]:
success = pd.concat([data_0[data_0['state'] == 'successful'], data_1[data_1['state'] == 'successful'], \
                     data_2[data_2['state'] == 'successful'], data_3[data_3['state'] == 'successful'], \
                     data_4[data_4['state'] == 'successful'], data_5[data_5['state'] == 'successful'], \
                     data_6[data_6['state'] == 'successful'], data_7[data_7['state'] == 'successful'], \
                     data_8[data_8['state'] == 'successful'], data_9[data_9['state'] == 'successful'], \
                     data_10[data_10['state'] == 'successful'], data_11[data_11['state'] == 'successful'], \
                    ])
fail = pd.concat([data_0[data_0['state'] == 'failed'], data_1[data_1['state'] == 'failed'], \
                     data_2[data_2['state'] == 'failed'], data_3[data_3['state'] == 'failed'], \
                     data_4[data_4['state'] == 'failed'], data_5[data_5['state'] == 'failed'], \
                     data_6[data_6['state'] == 'failed'], data_7[data_7['state'] == 'failed'], \
                     data_8[data_8['state'] == 'failed'], data_9[data_9['state'] == 'failed'], \
                     data_10[data_10['state'] == 'failed'], data_11[data_11['state'] == 'failed'], \
                    ])

In [197]:
print success.shape
print fail.shape

(17276, 32)
(4747, 32)


In [94]:
print fail['blurb'].head()
print success['blurb'].head()

20    You know you "toilet text”. Your phone may hav...
21    Track your gps ball from tees to trees. Christ...
22    A cellphone charger that is powered by the use...
23    Car Location Locator works with GPS signals an...
24    The revolutionary AiDrive features Certified L...
Name: blurb, dtype: object
0    Debauchery, laughter, violence and politics. W...
1    Bring a touring character education play about...
2    1984. An IRA bomb explodes at the Grand Hotel....
3    Zoe is a teenage girl growing up in a deeply d...
4    Theaterprojekt 12. Kl. Waldorfschule Essen. 2 ...
Name: blurb, dtype: object


In [200]:
def review_to_wordlist(i, raw, remove_stopwords=False, join_with_space=False):
        ''' Clean one review with optional removing stop words, & join the output with space. '''

        if pd.isnull(raw):
            print i, ':', raw
        else:
            letters_only = re.sub('[^a-zA-Z]', ' ', raw)
            words = letters_only.lower().split()
            # Make a set of stopwords
            if remove_stopwords:
                stop_words = set(stopwords.words("english"))
                meaningful_words = [w for w in words if not w in stop_words]
            if join_with_space:
                return ' '.join(meaningful_words)
            else:
                return words


def get_all_cleaned(raw, remove_stopwords=False, join_with_space=False):
    ''' Loop through all reviews, and preprocess all reviews. '''

    num_reviews = raw.size
    clean_reviews = []
    print "Cleaning and parsing the data set movie reviews...\n"
    for i, raw_item in enumerate(raw):
        if i%5000 == 0:
            print 'Review %d of %d\n' %(i, num_reviews)
        clean_reviews.append(review_to_wordlist(i, raw_item, \
                                                    remove_stopwords, \
                                                    join_with_space))
    return clean_reviews

In [201]:
print 'Check for total NaN values: ', sum(pd.isnull(success.iloc[:]['blurb']))
print 'Cleaning success blurb...'
success_blurbs = get_all_cleaned(success['blurb'], True, True)

print 'Cleaning fail blurb...'
fail_blurbs = get_all_cleaned(fail['blurb'], True, True)

Check for total NaN values:  0
Cleaning success blurb...
Cleaning and parsing the data set movie reviews...

Review 0 of 17276

Review 5000 of 17276

Review 10000 of 17276

Review 15000 of 17276

Cleaning fail blurb...
Cleaning and parsing the data set movie reviews...

Review 0 of 4747



In [202]:
print len(success_blurbs), len(fail_blurbs)
success_blurbs = [x for x in success_blurbs if x is not None]
fail_blurbs = [x for x in fail_blurbs if x is not None]
print len(success_blurbs), len(fail_blurbs)

print success_blurbs[0:2]
print fail_blurbs[0:2]

17276 4747
17276 4747
['debauchery laughter violence politics want help drama soc production posh best', 'bring touring character education play making wise choices elementary students kentuckiana vote kylie president']
['know toilet text phone may harmful germs solution cleaner cube stick need', 'track gps ball tees trees christmas birthdays fathers day corp charity tournies retirement best gift ever']


In [232]:
# Use tf-idf
success_tf = TfidfVectorizer(min_df=2, \
                            max_df=0.75, \
                            max_features=500, \
                            ngram_range=(1, 4), \
                            sublinear_tf=True)
success_tf_bow = success_tf.fit_transform(success_blurbs)
print success_tf_bow.shape

fail_tf = TfidfVectorizer(min_df=2, \
                            max_df=0.75, \
                            max_features=500, \
                            ngram_range=(1, 4), \
                            sublinear_tf=False)
fail_tf_bow = fail_tf.fit_transform(fail_blurbs)
print fail_tf_bow.shape

(17276, 500)
(4747, 500)


In [233]:
# Convert Scipy sparse matrix to Pandas dataframe

# For fail.
print type(fail_tf_bow)
df_tf_fail = pd.DataFrame([ pd.Series(fail_tf_bow[i].toarray().ravel()) \
                              for i in np.arange(fail_tf_bow.shape[0]) ])
print type(df_fail)

# For success.
df_tf_success = pd.DataFrame([ pd.Series(success_tf_bow[i].toarray().ravel()) \
                              for i in np.arange(success_tf_bow.shape[0]) ])
print type(df_success)

<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [234]:
top_ten_tf_fail = df_tf_fail.sum(axis=0).sort_values(ascending=False)
print top_ten_tf_fail
print top_ten_tf_fail.index[0:10]

top_ten_tf_success = df_tf_success.sum(axis=0).sort_values(ascending=False)
print top_ten_tf_success
print top_ten_tf_success.index[0:10]

476    121.925176
189    118.468475
281    106.918045
292     98.460310
303     92.197888
486     91.628225
394     88.961843
316     76.139932
252     74.280630
229     70.983085
88      70.371236
468     69.875782
291     68.192593
389     67.251572
172     67.015276
69      66.124189
154     65.383023
74      65.018564
153     64.401342
317     64.291433
157     62.927571
21      61.691744
275     58.898328
474     58.658889
274     58.477259
472     57.550103
433     57.107319
453     54.981952
429     54.733153
273     54.675699
          ...    
95       9.884521
492      9.880545
94       9.863569
241      9.859537
406      9.850527
168      9.836245
305      9.822836
245      9.733967
349      9.712438
8        9.706715
3        9.687465
307      9.644118
334      9.619756
298      9.617794
89       9.617036
81       9.574636
452      9.550662
87       9.542778
6        9.430909
283      9.399482
467      9.350732
170      9.242333
352      9.102966
421      9.063699
430      9

In [245]:
# Reverse dictionary for Fail, and print top 10 words.
print fail_tf.vocabulary_
new_tf_fail = {v: k for k, v in fail_tf.vocabulary_.iteritems()}
print '\n', new_tf_fail


# Reverse dictionary for Success, and print top 10 words.
print success_tf.vocabulary_
new_tf_success = {v: k for k, v in success_tf.vocabulary_.iteritems()}
print '\n', new_tf_success


print '\nTop 20 words for failed campaigns: '
fail_words = []
for index in top_ten_tf_fail.index[0:20]:
    print new_tf_fail[index]
    fail_words.append(new_tf_fail[index])
    
print '\nTop 20 words for successful campaigns: '
success_words = []
for index in top_ten_tf_success.index[0:20]:
    print new_tf_success[index]
    success_words.append(new_tf_success[index])

{u'bringing': 49, u'kids': 217, u'company': 76, u'learn': 224, u'help us': 190, u'video': 461, u'makes': 253, u'comedy': 71, u'books': 46, u'could': 86, u'become': 39, u'innovative': 207, u'production': 330, u'haunted': 187, u'develop': 107, u'exciting': 140, u'american': 17, u'personal': 309, u'get': 172, u'across': 2, u'services': 377, u'like': 232, u'today': 435, u'questions': 346, u'long': 242, u'gaming': 170, u'new': 281, u'best': 40, u'life': 229, u'ideas': 200, u'people': 303, u'current': 94, u'costs': 85, u'imagine': 201, u'set': 378, u'see': 371, u'community': 74, u'fund': 163, u'location': 240, u'actors': 5, u'secure': 370, u'global': 177, u'change': 61, u'dedicated': 104, u'en': 124, u'theater': 428, u'real': 351, u'read': 350, u'feel': 150, u'food': 156, u'run': 361, u'products': 331, u'tell': 425, u'stop': 408, u'arts': 30, u'audience': 31, u'entertainment': 128, u'idea': 199, u'interactive': 208, u'create': 88, u'pet': 310, u'works': 485, u'buy': 56, u'end': 125, u'record

In [252]:
for value in top_ten_tf_fail.head(20):
    print value

121.925175987
118.468475187
106.918045491
98.460309613
92.1978876336
91.6282248154
88.9618428571
76.1399319464
74.280630191
70.9830852637
70.3712361629
69.8757818774
68.1925931562
67.2515716739
67.0152757927
66.1241885523
65.3830234586
65.0185638036
64.4013421858
64.291432595


In [253]:
for value in top_ten_tf_success.head(20):
    print value

756.20876664
458.797596164
445.948779394
381.398791996
348.908268766
325.122689748
289.991846913
289.952590525
289.911820168
280.148523615
243.49546176
237.961274005
213.137740766
212.095107472
208.85845962
201.417217883
200.624505082
192.725743876
189.255978995
189.014211222


In [250]:
print np.sort(fail_words)
print np.sort(success_words)
repeated = 0
all_words = []
for fail in fail_words:
    for success in success_words:
        if fail == success:
            print fail, success
            repeated += 1
print repeated

[u'com' u'community' u'create' u'find' u'first' u'get' u'help' u'life'
 u'make' u'new' u'one' u'online' u'people' u'platform' u'play' u'site'
 u'social' u'want' u'website' u'world']
[u'art' u'book' u'children' u'first' u'get' u'help' u'life' u'love' u'make'
 u'music' u'new' u'novel' u'one' u'people' u'project' u'stories' u'story'
 u'time' u'us' u'world']
help help
new new
people people
world world
make make
life life
one one
get get
first first
9


In [209]:
# Bag of Words using count.

print "Creating the bag of words for train data...\n"
success_vectorizer = CountVectorizer(analyzer='word', \
                            tokenizer=None, \
                            preprocessor=None, \
                            stop_words=None, \
                            max_features=500)

fail_vectorizer = CountVectorizer(analyzer='word', \
                            tokenizer=None, \
                            preprocessor=None, \
                            stop_words=None, \
                            max_features=500)

success_bag_of_words = success_vectorizer.fit_transform(success_blurbs)
# print success_vectorizer.vocabulary_
print success_bag_of_words.shape
print '\n'

fail_bag_of_words = fail_vectorizer.fit_transform(fail_blurbs)
# print fail_vectorizer.vocabulary_
print fail_bag_of_words.shape

Creating the bag of words for train data...

(17276, 500)


(4747, 500)


In [210]:
# Convert Scipy sparse matrix to Pandas dataframe

# For fail.
print type(fail_bag_of_words)
df_fail = pd.DataFrame([ pd.Series(fail_bag_of_words[i].toarray().ravel()) \
                              for i in np.arange(fail_bag_of_words.shape[0]) ])
print type(df_fail)

# For success.
df_success = pd.DataFrame([ pd.Series(success_bag_of_words[i].toarray().ravel()) \
                              for i in np.arange(success_bag_of_words.shape[0]) ])

<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.frame.DataFrame'>


In [211]:
# print df_temp.sum(axis=0)
top_ten_fail = df_fail.sum(axis=0).sort_values(ascending=False)
print top_ten_fail.index[0:10]

top_ten_success = df_success.sum(axis=0).sort_values(ascending=False)
print top_ten_success.index[0:10]

Int64Index([193, 478, 286, 294, 398, 306, 488, 320, 258, 87], dtype='int64')
Int64Index([56, 206, 297, 488, 164, 415, 27, 78, 304, 249], dtype='int64')


In [213]:
# Reverse dictionary for Fail, and print top 10 words.
print fail_vectorizer.vocabulary_
new_fail = {v: k for k, v in fail_vectorizer.vocabulary_.iteritems()}
print '\n', new_fail

# Reverse dictionary for Success, and print top 10 words.
print success_vectorizer.vocabulary_
new_success = {v: k for k, v in success_vectorizer.vocabulary_.iteritems()}
print '\n', new_success

print '\nTop 10 words for failed campaigns: '
for index in top_ten_fail.index[0:10]:
    print new_fail[index]
print '\nTop 10 words for successful campaigns: '
for index in top_ten_success.index[0:10]:
    print new_success[index]

{u'bringing': 48, u'kids': 222, u'music': 279, u'care': 57, u'brings': 49, u'want': 472, u'travel': 447, u'gaming': 171, u'skills': 394, u'series': 379, u'perform': 308, u'service': 380, u'worlds': 489, u'project': 341, u'person': 311, u'best': 39, u'much': 278, u'pro': 332, u'location': 246, u'life': 235, u'child': 62, u'things': 435, u'big': 41, u'projects': 342, u'online': 294, u'possible': 325, u'unique': 455, u'right': 365, u'old': 292, u'people': 306, u'easy': 121, u'creative': 90, u'support': 420, u'offer': 290, u'talents': 425, u'time': 437, u'connecting': 78, u'team': 427, u'current': 92, u'love': 252, u'working': 486, u'data': 97, u'st': 407, u'years': 497, u'course': 86, u'internet': 215, u'training': 446, u'one': 293, u'vote': 471, u'custom': 93, u'open': 295, u'city': 65, u'future': 167, u'take': 422, u'going': 183, u'jobs': 218, u'many': 264, u'wants': 473, u'photos': 317, u'technology': 428, u'costs': 84, u'summer': 419, u'actors': 5, u'around': 25, u'dark': 96, u'world'